In [1]:
import pandas as pd
import os

In [95]:
results_dir='~/data/results/twconvrecsys/twconvrecsys_results_twconv_2016_usersec_cftopics'
results_dir=os.path.expanduser(results_dir)

In [96]:
fpath = os.path.join(results_dir, 'twconv_2016_usersec_dialogs.data_0.1_0.5_10_10_0.75_200_0_score.out')
ds = pd.read_csv(fpath, sep='\t', names=['user_id','conv_id','y_pred','y_true'])
ds.shape, ds.conv_id.nunique(), ds.user_id.nunique()

((47095, 4), 3464, 4234)

In [97]:
ds.head()

,user_id,conv_id,y_pred,y_true
0,4,1,0.027162,1
1,15,8,0.902758,1
2,19,16,-0.017417,1
3,21,19,0.207134,1
4,25,23,-0.005840,1


In [98]:
users = ds[ds.y_true==1].user_id.unique()
users

array([   4,   15,   19, ..., 4228, 4180, 4160])

In [99]:
groups = ds[ds.user_id.isin(users)].groupby('user_id')

test_data = []
predictions = []

for user_id, user_scores in groups:
    distractors = user_scores[user_scores.y_true==0].head(9)
    ground = user_scores[user_scores.y_true==1]

    for ix, row in ground.iterrows():
        
        instance = [user_id]
        instance.append(row['conv_id'])
        instance.extend(distractors.conv_id.values)
        label = 0#list(user_scores.y_true.values).index(1)
        instance.append(label)
        test_data.append(instance)
        preds = [row['y_pred']]
        preds.extend( distractors.y_pred.values )
        predictions.append(preds)

In [100]:
len(test_data), len(predictions)

(4755, 4755)

In [101]:
data_dir='~/data/twconv/twconv_2016_usersec/cft'
data_dir=os.path.expanduser(data_dir)
os.makedirs(data_dir, exist_ok=True)


In [102]:
cols='source,target0,target1,target2,target3,target4,target5,target6,target7,target8,target9,label'
cols=cols.split(',')
test_ds = pd.DataFrame(test_data, columns=cols)
fpath = os.path.join(data_dir, 'train.csvrecords')
test_ds.to_csv(fpath, index=False)
fpath = os.path.join(data_dir, 'valid.csvrecords')
test_ds.to_csv(fpath, index=False)
fpath = os.path.join(data_dir, 'test.csvrecords')
test_ds.to_csv(fpath, index=False)
test_ds.shape

(4755, 12)

In [103]:
results_dir='~/data/results/twconvrecsys/twconv_2016_users_v3/cft'
results_dir=os.path.expanduser(results_dir)
os.makedirs(results_dir, exist_ok=True)
fpath = os.path.join(results_dir, 'predictions.csv')
pred_ds = pd.DataFrame(predictions)
pred_ds.to_csv(fpath, index=False, header=None)
test_ds.shape

(4755, 12)